In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
import time
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import date
import re
import os
import warnings
warnings.filterwarnings('ignore')

### Atualização de temporada

In [2]:
def clean_name(name):
    name = re.sub(r'\s*-.*$', '', name.strip())  # Remover informações extras
    return name.title()  # Caso contrário, padronizar nome com letras maiúsculas iniciais

In [3]:
def find_point_five(number):
    # Converte o número em uma string
    number_str = str(number)
    # Verifica se a string termina com ".5"
    if number_str.endswith(".5"):
        return True
    else:
        return False

In [4]:
lista_ligas = ['Eua Nhl']
hoje = date.today()

dia = "hoje"

## Scraping dos jogos do dia

In [5]:
# Instanciando o Objeto ChromeOptions
options = webdriver.EdgeOptions()

# Passando algumas opções para esse ChromeOptions
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Criação do WebDriver do Chrome
wd_Chrome = webdriver.Edge(options=options)

# Com o WebDrive a gente consegue a pedir a página (URL)
wd_Chrome.get("https://www.flashscore.com.br/basquete/")
time.sleep(2)

# Para jogos do dia seguinte
if dia == "amanha":
    wd_Chrome.find_element(By.XPATH,'//*[@id="live-table"]/div[1]/div[2]/div/button[3]').send_keys(Keys.ENTER)
    time.sleep(2)

# Pegando o ID dos Jogos
id_jogos = []
jogos = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.event__match--scheduled')

for i in jogos:
    id_jogos.append(i.get_attribute("id"))

# Exemplo de ID de um jogo: 'g_1_Gb7buXVt'    
id_jogos = [i[4:] for i in id_jogos]

base_jogos = pd.DataFrame({})

for link in tqdm(id_jogos, total=len(id_jogos)):
    wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/resumo-de-jogo/resumo-de-jogo')
    
    Odds_H = 0
    Odds_A = 0
    Over_Line = 0
    Odds_Over = 0
    Odds_Under = 0
    HA_Line = 0
    HA_Odds_H = 0
    HA_Odds_A = 0
    
    # Pegando as Informacoes Básicas do Jogo
    try:
        Date = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[0]
        Time = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[1]
        Country = wd_Chrome.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country').text.split(':')[0]
        League = wd_Chrome.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country')
        League = League.find_element(By.CSS_SELECTOR,'a').text
        Home = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__home')
        Home = Home.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
        Away = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__away')
        Away = Away.find_element(By.CSS_SELECTOR,'div.participant__participantName').text

        Country = clean_name(Country)
        League = clean_name(League)
        League_name = Country + ' ' + League

        # if League_name not in lista_ligas:
        #     # Condição satisfeita, pula para a próxima iteração do loop
        #     continue
    
        # Match Odds
        wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/comparacao-de-odds/home-away/tr-incluindo-prol')
        time.sleep(1)
        celulas = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
        
        for celula in celulas:
            bookie = celula.find_element(By.CSS_SELECTOR,'img.prematchLogo')
            bookie = bookie.get_attribute('title')
            if ((bookie == 'bet365')):
                Odds_H = float(celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                Odds_A = float(celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
                break
            else:
                pass


        # Over/Under
        wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/comparacao-de-odds/acima-abaixo/tr-incluindo-prol')
        time.sleep(1)
        celulas = wd_Chrome.find_elements(By.CLASS_NAME,'ui-table__row')

        for celula in celulas:
            bookie = celula.find_element(By.CSS_SELECTOR, 'img.prematchLogo').get_attribute('title')
            Odds_Over = float(celula.find_elements(By.TAG_NAME,'span')[1].text)
            Over_Line = float(celula.find_elements(By.TAG_NAME,'span')[0].text)
            if (((bookie == 'bet365') and (Odds_Over >= 1.80) and find_point_five(Over_Line))):
                Odds_Under = float(celula.find_elements(By.TAG_NAME,'span')[2].text)
                break
            else:
                Over_Line, Odds_Over, Odds_Under = 0, 0, 0                   
                pass


        # Handicap
        wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/comparacao-de-odds/handicap-asiatico/tr-incluindo-prol')
        time.sleep(1)
        celulas = wd_Chrome.find_elements(By.CLASS_NAME,'ui-table__row')

        for celula in celulas:
            
            HA_Line = float(celula.find_elements(By.TAG_NAME,'span')[0].text)
            HA_Odds_H = float(celula.find_elements(By.TAG_NAME,'span')[1].text)

            bookie = celula.find_element(By.CSS_SELECTOR, 'img.prematchLogo').get_attribute('title')
            if ((bookie == 'bet365' and HA_Odds_H >= 1.80) and (bookie == 'bet365' and HA_Odds_H <= 2.10) and (find_point_five(HA_Line))):
                HA_Odds_A = float(celula.find_elements(By.TAG_NAME,'span')[2].text)
                break
            else:
                pass

        # print(Date, Home, Over_Line, Odds_Over, Odds_Under, HA_Line, HA_Odds_H, HA_Odds_A)
        # print()

        base_jogos.loc[base_jogos.shape[0],['Fixture ID', 'Date','League','Time','Home','Away','Odds_H','Odds_A','Over_Line','Odds_Over','Odds_Under','HA_Line','HA_Odds_H','HA_Odds_A']] = [
            link, Date, League_name, Time, Home, Away, Odds_H, Odds_A, Over_Line, Odds_Over, Odds_Under, HA_Line, HA_Odds_H, HA_Odds_A
        ]
    except:
        continue

wd_Chrome.quit()

The msedgedriver version (117.0.2045.31) detected in PATH at c:\Users\johnn\OneDrive\Documentos\apostas\basketball_models\msedgedriver.exe might not be compatible with the detected MicrosoftEdge version (118.0.2088.57); currently, msedgedriver 118.0.2088.57 is recommended for MicrosoftEdge 118.*, so it is advised to delete the driver in PATH and retry
100%|██████████| 23/23 [02:05<00:00,  5.45s/it]


In [6]:
backup = base_jogos.copy()

base_jogos = base_jogos[base_jogos['Odds_H'] != 0]
base_jogos.reset_index(drop=True, inplace=True)

# Ajusta coluna de data
base_jogos['Date'] = pd.to_datetime(base_jogos['Date'], format='%d.%m.%Y')
base_jogos.to_csv(f'jogos_do_dia/{hoje}.csv', index=False)
base_jogos

,Fixture ID,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
0,xCCpqtTD,2023-10-20,Eua Nba,20:00,Indiana Pacers,Cleveland Cavaliers,2.10,1.77,224.5,1.91,1.91,1.5,1.91,1.91
1,M3UcLxy7,2023-10-20,Eua Nba,20:00,Philadelphia 76ers,Atlanta Hawks,1.80,2.05,220.5,1.91,1.91,-2.5,1.91,1.91
2,StVgMIL0,2023-10-20,Eua Nba,20:30,Toronto Raptors,Washington Wizards,1.45,2.80,219.5,1.91,1.91,-5.5,1.91,1.91
3,G6RkNb6f,2023-10-20,Eua Nba,21:00,Dallas Mavericks,Detroit Pistons,1.50,2.70,223.5,1.91,1.91,-5.5,1.91,1.91
4,YsxoOvil,2023-10-20,Eua Nba,21:00,Houston Rockets,Miami Heat,1.69,2.25,219.5,1.91,1.91,-2.5,1.91,1.91
5,fiwsPKyr,2023-10-20,Eua Nba,21:00,Milwaukee Bucks,Memphis Grizzlies,1.57,2.55,223.5,1.91,1.91,-4.5,1.91,1.91
6,2aoFT0TQ,2023-10-20,Eua Nba,23:00,Golden State Warriors,San Antonio Spurs,1.45,2.80,226.5,1.91,1.91,-5.5,1.91,1.91
7,QHWpUs1a,2023-10-20,Argentina Liga A,21:00,Argentino,Obras,2.30,1.56,157.5,1.83,1.83,3.5,1.83,1.83
8,48VlT1G5,2023-10-20,Argentina Liga A,22:00,Independiente de Oliva,San Martin,2.00,1.71,159.5,1.83,1.83,1.5,1.83,1.83
9,SYluDQqC,2023-10-20,Japão B.League,23:55,Hiroshima D.,Utsunomiya Brex,2.30,1.56,146.5,1.83,1.83,1.5,1.83,1.83
